In [12]:
from py2cfg import CFGBuilder
import numpy as np
import networkx as nx 
import graphviz
import matplotlib.pyplot as plt 
import re

In [13]:
def ParseGraph(cfg):

    i = -1
    nodes = []
    for block in cfg:
            current = []
            neighbours = []
            source = str(i) +" : "+ str(block.get_source())
            #if str(block.get_source()).split(" ")[0] in ["for","while","if","return"]:
            #    source = str(i) +" : "+ str(block.get_source()).split(" ")[0]
            #else:
            #    source = str(i)
            source = source.strip("\n")
            #print (source)
            #print("block")
            #print(block)
            node = re.findall("\d*@", str(block))[0]
            node = int(node[:-1])
            if (node != "1"):
                #print("node:", node)
                for e in block.exits:
                    #print(e)
                    neighbour = re.findall("\d*@", str(e))[1]
                    #print("neighbour:", neighbour)
                    neighbour = int(neighbour[:-1])
                    #print("neighbour:", neighbour)
                    neighbours.append(neighbour)
            current.append(i)
            current.append(node)
            current.append(neighbours)
            current.append(source)
            nodes.append(current)
            i += 1
            #print("\n")

            # nodes contem as seguintes informações para cada node: 
            #                                                       node[0] : o index novo a ser utilizado para o nó atual
            #                                                       node[1] : o index antigo configurado pelo py2cfg
            #                                                       node[2] : a lista de nós filhos do nó atual
            #                                                       node[3] : trecho de codigo que esta atrelado à string
        
    #remove o primeiro nó que faz a definição de função, este no não é conecta a arvore principal
    nodes.pop(0)
    #print(nodes)

    return nodes

In [14]:
def FormatGraph(nodes):
	
	#gera um dicionario que sera usado para substituir os valores do py2cfg por numeros em ordem crescente, 
    # o metodo do py2cfg corta nos nos redundantes, deixando referencias fantasmas no grafo original, necessitando essa limpeza da saida para podermos manipular

    dict = {}
    for node in nodes:
        dict[node[1]] = node[0]
    print("dictionary:", dict)

    #substitui os nós vizinhos de cada nó usando o dicionario
    for node in nodes:
        for idx, neighbour in enumerate(node[2]):
            node[2][idx] = dict[neighbour]

    # remove o agora redundante valores antigos de index do py2cfg
    for node in nodes:
        node.pop(1)

    # resultado final, uma lista de nós, contendo seus vizinhos e o tamanho do grafo
    print("novo grafo: ",nodes)
    print("quantidade de nós:",len(nodes))

    return nodes

In [15]:
def GerarRequisitos(nodes):
	
    R1 = []
    R2 = []
    R3 = []

	# gera primeira lista de requisitos de nós, percorendo a lista e registrando os nós existentes 
    for node in nodes:
        R1.append(node[0])

    # gera lista de requisitos de arcos, percorendo a lista e explorando os vizinhos
    for node in nodes:
        for idx, neighbour in enumerate(node[1]):
            R2.append([node[0],node[1][idx]])

    # gera lista de requisitos de par de arcos, percorendo a lista e explorando os vizinhos
    for node in nodes:
        #print("current node:",node[0])
        for idx1, neighbour1 in enumerate(node[1]):
            #print("current son:",neighbour1)
            #print("grandsons:", str(nodes[neighbour1][1]))
            if len(nodes[neighbour1][1]) == 0:
                if len(R3) == 0:
                    #print("nó isolado, adicionando arco como requisito", str([node[0],node[1][idx1]]))
                    R3.append([node[0],node[1][idx1]])
            for idx2, neighbour2 in enumerate(nodes[neighbour1][1]):
                #print("current grandson:",neighbour2)
                R3.append([node[0],node[1][idx1],nodes[neighbour1][1][idx2]])
    print("requisitos de nós:",R1)
    print("requisitos de arcos:",R2)
    print("requisitos de par de arcos:",R3)

    return [R1,R2,R3]

In [16]:
def CreateGraphviz(nodes):

	G = graphviz.Digraph(node_attr={'shape':'ellipse'},format='svg')

	for node in nodes:
		i = 0
		if "if" in str(node[2]):
			G.node(str(node[0]),label = node[2], shape = "diamond", color = "red")
		elif "while" in str(node[2]):
			G.node(str(node[0]),label = node[2], shape = "polygon", color = "orange")
		elif "for" in str(node[2]):
			G.node(str(node[0]),label = node[2], shape = "octagon", color = "orange")
		elif "try" in str(node[2]):
			G.node(str(node[0]),label = node[2], shape = "Mdiamond", color = "orange")
		elif "return" in str(node[2]):
			G.node(str(node[0]),label = node[2], shape = "parallelogram", color = "green")
		else:
			G.node(str(node[0]),node[2])
		for son in node[1]:
			if i == 0 :
				if len(node[1]) == 2:
					G.edge(str(node[0]), str(son), color = "green")
				else:
					G.edge(str(node[0]), str(son), color = "black")
			elif i == 1 :
				G.edge(str(node[0]), str(son), color = "red")
			else:
				G.edge(str(node[0]), str(son))
			i+= 1

	return G

In [17]:
def CreatePath(nodes, path, tests, requisitos, limit, depth):

	if depth > limit:
		return tests

	if path == []:
		for element in nodes:
			if not element[1]:
				path.insert(0, element[0])
				depth = depth + 1
				tests = CreatePath(nodes, path, tests, requisitos, limit, depth)
				depth = depth - 1
				path.pop(0)

	elif path[0] == 0:
		i = 0
		new = False
		while i+2 < len(path):
			requisito = [path[i],path[i+1],path[i+2]]
			if requisito in requisitos:
				new = True
				requisitos.remove(requisito)
			i += 1
		if len(path) == 2:
			requisito = [path[i],path[i+1]]
			if requisito in requisitos:
				new = True
				requisitos.remove(requisito)
		if new:
			tests.append(path.copy())
	
	elif path[0] != 0:
		for element in nodes:
			if path[0] in element[1]:
				path.insert(0, element[0])
				depth = depth + 1
				tests = CreatePath(nodes, path, tests, requisitos, limit, depth)
				depth = depth - 1
				path.pop(0)
	return tests



In [18]:
def SuggestTest(nodes, requisitos, limit):
	requisitosFaltantes = requisitos.copy()
	requisitosTestes = requisitos.copy()
	tests = []
	compliance = []
	while requisitosTestes:
		tests = CreatePath(nodes, [], tests, requisitosTestes, limit, 0)
		limit = limit+1

	for test in tests:
			compliancetemp = []
			i = 0
			while i+2 < len(test):
				requisito = [test[i],test[i+1],test[i+2]]
				print(requisito)
				if requisito in requisitosFaltantes:
					compliancetemp.append(requisito)
					requisitosFaltantes.remove(requisito)
				i += 1
			if len(test) == 2:
				requisito = [test[i],test[i+1]]
				if requisito in requisitosFaltantes:
					compliancetemp.append(requisito)
					requisitosFaltantes.remove(requisito)
			compliance.append(compliancetemp)

	return tests, compliance
	
	


In [19]:
def GeradorDeRequisitos(filepath):
    cfg = CFGBuilder().build_from_file('example', filepath)
    cfg.build_visual('exampleCFG', 'svg')

    #começa pre processamento do grafo gerado pelo py2cfg para o formato de uma lista

    nodes = ParseGraph(cfg)

    nodes = FormatGraph(nodes)

    requisitos = GerarRequisitos(nodes)

    requisitosTestes = requisitos.copy()

    Graph = CreateGraphviz(nodes)
    
    name = "requisitos " + filename +".txt" 

    testes, compliance= SuggestTest(nodes, requisitosTestes[2], 3)

    requisitos[0] = "requisitos de nós (" + str(len(requisitos[0])) + ") : " + str(requisitos[0])

    requisitos[1] = "requisitos de arcos (" + str(len(requisitos[1])) + ") : " + str(requisitos[1])

    requisitos[2] = "requisitos de par de arcos (" + str(len(requisitos[2])) + ") : " + str(requisitos[2])

    testesPropostos = []
    i = 0
    for teste in testes:
        testesPropostos.append("T"+ str(i) + ": " + str(teste))
        i += 1


    open(name, 'w').write(str(requisitos[0]))
    open(name, 'a').write('\n')
    open(name, 'a').write(str(requisitos[1]))
    open(name, 'a').write('\n')
    open(name, 'a').write(str(requisitos[2]))
    open(name, 'a').write('\n')
    open(name, 'a').write('\n')

    i = 0
    for proposta in testesPropostos:
        open(name, 'a').write(str(proposta))
        open(name, 'a').write('\n')
        open(name, 'a').write(str(compliance[i]))
        open(name, 'a').write('\n')
        open(name, 'a').write('\n')
        i += 1

    print(Graph.source)
    return Graph

In [20]:
filename = "primo"
entry = "./"+filename+".py"

G = GeradorDeRequisitos(entry)
G.render(filename, view = True)


(dot.exe:18912): Pango-WARNING **: couldn't load font "DejaVu Sans Mono Not-Rotated 14", falling back to "Sans Not-Rotated 14", expect ugly output.


dictionary: {3: 0, 4: 1, 6: 2, 7: 3, 5: 4, 12: 5, 13: 6, 14: 7}
novo grafo:  [[0, [1, 2], '0 : if n == 1:'], [1, [], '1 : return False'], [2, [3, 4], '2 : if n == 2:'], [3, [], '3 : return True'], [4, [5, 6], '4 : for contador in range(2, n):'], [5, [7, 4], '5 : if n % contador == 0:'], [6, [], '6 : return True'], [7, [], '7 : return False']]
quantidade de nós: 8
requisitos de nós: [0, 1, 2, 3, 4, 5, 6, 7]
requisitos de arcos: [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [4, 6], [5, 7], [5, 4]]
requisitos de par de arcos: [[0, 1], [0, 2, 3], [0, 2, 4], [2, 4, 5], [2, 4, 6], [4, 5, 7], [4, 5, 4], [5, 4, 5], [5, 4, 6]]
[0, 2, 3]
[0, 2, 4]
[2, 4, 6]
[0, 2, 4]
[2, 4, 5]
[4, 5, 4]
[5, 4, 6]
[0, 2, 4]
[2, 4, 5]
[4, 5, 4]
[5, 4, 5]
[4, 5, 4]
[5, 4, 6]
[0, 2, 4]
[2, 4, 5]
[4, 5, 7]
digraph {
	node [shape=ellipse]
	0 [label="0 : if n == 1:" color=red shape=diamond]
	0 -> 1 [color=green]
	0 -> 2 [color=red]
	1 [label="1 : return False" color=green shape=parallelogram]
	2 [label="2 : if n == 2:" colo

'primo.svg'